In [77]:
import gmplot
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt

In [78]:
#Luetaan data
data = pd.read_csv('./Mittaus 1/Location.csv', delimiter=",", decimal=".")
data.head()

,time,seconds_elapsed,bearingAccuracy,speedAccuracy,verticalAccuracy,horizontalAccuracy,speed,bearing,altitude,longitude,latitude
0,1696508108019665400,3.703666,10,20,34.216305,7.00,0.146520,234.195694,36.600002,25.516165,65.071375
1,1696508109655879200,5.339879,10,20,34.232666,7.00,1.091845,71.881088,36.600002,25.516227,65.071393
2,1696508109996000000,5.680000,10,20,5.733375,7.00,1.169993,69.123779,36.600002,25.516240,65.071398
3,1696508110627351000,6.311351,10,20,5.739688,6.75,1.204262,78.989891,36.600002,25.516264,65.071394
4,1696508111639929000,7.323929,10,20,5.749815,6.40,0.847965,75.783852,36.600002,25.516267,65.071396


In [79]:
#Määritellään funktio (Haversinen kaava), joka ottaa sisään alku- ja loppupisteen koordinaantit lon1, lat1, lon2, lat2
#ja palauttaa niiden välisen matkan
long1 = data['longitude'].iloc[0]

long2 = data['longitude'].iloc[-1]

lati1 = data['latitude'].iloc[0]

lati2 = data['latitude'].iloc[-1]



def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points  on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers.
    return c * r #Palauttaa tämän

distancetraveled = haversine(long1, lati1, long2, lati2)*1000
print('You traveled',round(distancetraveled), 'meters')

You traveled 202 meters


In [80]:
#Lasketaan nopeus Haversinen kaavaa käyttäen (vaikka se olisi jo valmiiksi datassa)

lat = data['latitude'] #Määritellään parametri lat, joka sisältää latitudin arvot
lon = data['longitude'] #Määritellään parametri lon, joka sisältää longitudin arvot

data['dist'] = np.zeros(lat.shape[0]) #Lisätään dataan nollasarake, johon kohta lasketaan pisteiden välinen etäisyys
data['time_diff']  = np.zeros(lat.shape[0]) #Lisätään dataan nollasarake, johon kohta lasketaan pisteiden välinen aika

for i in range(lat.shape[0]-1): #For-luuppi käy läpi datan toiseksi viimeiselle riville saakka
  data['dist'][i] = haversine(lon[i], lat[i],lon[i+1], lat[i+1])*1000 #Datan pisteiden i ja i + 1 välimatka metreissä
  data['time_diff'][i] = data['seconds_elapsed'][i+1] - data['seconds_elapsed'][i]  #Peräkkäisten ajanhetkien i ja i+1 välimatka datassa


data['dist'][0] = 0
data['velocity'] = data['dist']/data['time_diff']

data.head()

C:\Users\janit\AppData\Local\Temp\ipykernel_11124\3706817946.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dist'][i] = haversine(lon[i], lat[i],lon[i+1], lat[i+1])*1000 #Datan pisteiden i ja i + 1 välimatka metreissä
C:\Users\janit\AppData\Local\Temp\ipykernel_11124\3706817946.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['time_diff'][i] = data['seconds_elapsed'][i+1] - data['seconds_elapsed'][i]  #Peräkkäisten ajanhetkien i ja i+1 välimatka datassa
C:\Users\janit\AppData\Local\Temp\ipykernel_11124\3706817946.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

,time,seconds_elapsed,bearingAccuracy,speedAccuracy,verticalAccuracy,horizontalAccuracy,speed,bearing,altitude,longitude,latitude,dist,time_diff,velocity
0,1696508108019665400,3.703666,10,20,34.216305,7.00,0.146520,234.195694,36.600002,25.516165,65.071375,0.000000,1.636214,0.000000
1,1696508109655879200,5.339879,10,20,34.232666,7.00,1.091845,71.881088,36.600002,25.516227,65.071393,0.810401,0.340121,2.382686
2,1696508109996000000,5.680000,10,20,5.733375,7.00,1.169993,69.123779,36.600002,25.516240,65.071398,1.171329,0.631351,1.855274
3,1696508110627351000,6.311351,10,20,5.739688,6.75,1.204262,78.989891,36.600002,25.516264,65.071394,0.229699,1.012578,0.226845
4,1696508111639929000,7.323929,10,20,5.749815,6.40,0.847965,75.783852,36.600002,25.516267,65.071396,0.722069,1.005767,0.717928


In [81]:
finaltime = data['seconds_elapsed'].iloc[-1]

def avgspeed(time, dist):
    hours = time / 3600
    kilometers = dist / 1000
    return kilometers/hours
    
averagespeed=avgspeed(finaltime,distancetraveled)
print('Average speed is',round(averagespeed,1),'km/h')



Average speed is 5.4 km/h


In [82]:
# Plot path in Google map

gmap = gmplot.GoogleMapPlotter(65.071375, 25.516165, 16, apikey="")

location_frame = data[['latitude','longitude']] #Luodaan Pandas data frame, jossa sarakkeina latitudi ja longitudi

location_list = location_frame.values.tolist() #Luodaan framesta lista

my_path = zip(*location_list)

gmap.plot(*my_path, color='cornflowerblue', edge_width=10)

gmap.draw('reitti1.html') #Tallennetaan tiedostoon